# RAG con Llama

## Iniciación del modelo de Ollama

In [1]:
from langchain_community.llms import Ollama #se utiliza para interactuar con modelos Ollama

MODEL_NAME = "llama3.2"

llm = Ollama(model = MODEL_NAME) #inicializamos el modelo de Ollama
response = llm.invoke("¡hola!") 

print(response)

C:\Users\Vicente García\AppData\Local\Temp\ipykernel_73184\839886171.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = MODEL_NAME) #inicializamos el modelo de Ollama


¡hola! ¿En qué puedo ayudarte hoy?


## Carga de la información de diferentes fuentes

In [2]:
from langchain_community.document_loaders import PyMuPDFLoader #para cargar documentos PDF

loader = PyMuPDFLoader("recursos/paper.pdf")
data_pdf = loader.load() #lee el archivo PDF y lo convierte en una lista de objetos Document
print(len(data_pdf)) #cada elemento de la lista data_pdf representa una página del PDF
print(data_pdf[0])

6
page_content='17
I.S.S.N. 0717 - 2079
CIENCIA Y ENFERMERIA X (1): 17-21, 2004
ELABORACIÓN DE UN ARTÍCULO CIENTÍFICO DE INVESTIGACIÓN
RESEARCH SCIENTIFIC ARTICLE: KNOW HOW
ELENA HENRÍQUEZ FIERRO* y MARIA INÉS ZEPEDA GONZALEZ**
RESUMEN
El artículo presenta la forma de redactar correctamente un artículo científico como reporte de una investigación,
contempla cada una de las etapas que debe contener para su aprobación. Se sugiere la forma de redactar desde el título
hasta la bibliografía, en un lenguaje comprensible y científico. Enfatiza en cómo deben ser presentados los resultados
obtenidos para su mejor comprensión de la comunidad científica.
Palabras claves: Artículo científico, producción, investigación.
ABSTRACT
This article introduces ways of writing correctly an article as a research report. It includes every step that has to be
taken into account for its approval. The authors provide suggestions of the report writing going from the title to the
references, both, in a comprehensi

## Creación de fragmentos

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter #se encarga de dividir elementos grandes y fragmentos más pequeños, teniendo en cuenta el texto para tratar de crear fragmentos coherentes

#esto se hace porque los LLM tienen un límite de la cantidad de texto que pueden procesar y porque ayuda al procesamiento interno por parte de los modelos
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500) #el tamaño máximo de cada fragmento es 2000 caracteres en este caso. Se superpondrán 500 caracteres entre fragmentos consecutivos, para ayudar a mantener el contexto y la coherencia entre los fragmentos
docs = text_splitter.split_documents(data_pdf) #se realiza la división de fragmentos

print(len(docs))

13


## Creación de embeddings

In [4]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings #una clase para crear embeddings

embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") #es un modelo preentrenado para generar embeddings de texto

print(embed_model)

model_name='sentence-transformers/all-MiniLM-L6-v2' max_length=512 cache_dir=None threads=None doc_embed_type='default' batch_size=256 parallel=None model=<fastembed.text.text_embedding.TextEmbedding object at 0x0000024E56AF65D0>


## Carga de los datos en Chroma

In [5]:
from langchain_community.vectorstores import Chroma #implementación de un almaceén de vectores que utiliza la base de datos ChromeDB internamente

vs = Chroma.from_documents( #crea el almacén de vectores 
    documents = docs, #necesita la lista de documentos  de entrada
    embedding = embed_model, #necesita el modelo de embeddings que se va a utilizar para convertir los documentos en vectores
    persist_directory = "chroma_db_dir", #necesita una ruta para guardar los datos del almacén de vectores y así persistirlos
    collection_name = "my_test" #necesita que se le proporcione un nombre a la colección para identificarla de otras
)

retriever = vs.as_retriever(search_kwargs={'k':3}) #retriever se utiliza para buscar documentos en el almacén de vectores. Con k : 3 le decimos que recupere los 3 elementos más similares (k vecinos) para cada consulta

## Plantilla de respuesta

In [6]:
from langchain.prompts import PromptTemplate #clase para crear plantillas de prompts 

#utilizamos una variable para darle contexto a la pregunta 
#utilizamos otra variable para colocar la pregunta que ha hecho el usuario
#con las plantillas podemos reutilizar código. Una vez que la plantilla está cubierta esto es lo que le llega al LLM
custom_prompt_template = """Usa la siguiente información para responder:

Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta correcta y nada más.
"""

prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])

## Pruebas

In [7]:
from langchain.chains import RetrievalQA #permite recuperar información de la base de datos

qa = RetrievalQA.from_chain_type(llm = llm,
                                chain_type = "stuff", #la información recuperada se meterá directamente en el prompt que se le envía al LLM (variable context). También se meterá la pregunta (query)
                                retriever = retriever,
                                return_source_documents = True, #para entender de dónde viene la información
                                chain_type_kwargs = {"prompt":prompt}) #la plantilla que se le pasó al inicio

questions = [{"query": "¿qué secciones tiene un artículo científico?"},
             {"query": "¿qué importancia tiene la sección de resultados en un artículo?"},
             {"query": "¿Cuántos artículos científicos se publicaron al año en 2012?"}]

for question in questions:
    response = qa.invoke(question) #query se traducirá internamente en la plantilla por question
    print(response["result"])

I y IV.
La sección de resultados es fundamental en un artículo científico, ya que es donde se presentan los hallazgos del estudio, las conclusiones y los datos obtenidos. Es importante que los resultados sean claros, concisos y precisos, y que estén presentados de manera ordenada y fácil de entender para la comunidad científica.
No se proporciona ninguna información para responder a esta pregunta.
